In [72]:
from pyspark.sql import SparkSession

In [73]:
from pyspark.sql.functions import *

In [74]:
from datetime import datetime

In [75]:
spark = SparkSession.builder.master('local[*]').appName('Terminal with sql').getOrCreate()

In [53]:
spark_version = spark.version 
print(f"Spark Version: {spark_version}")

Spark Version: 3.5.3


In [76]:
terminal_df = spark.read.options(header=True, inferSchema=True).csv('/home/maryam/content/lax_passengers_header.csv')

In [77]:
terminal_df.show(10)

+--------------------+--------------------+-----------------+-----------------+----------------------+---------------+
|     DataExtractDate|        ReportPeriod|         Terminal|Arrival_Departure|Domestic_International|Passenger_Count|
+--------------------+--------------------+-----------------+-----------------+----------------------+---------------+
|05/01/2014 12:00:...|01/01/2006 12:00:...|Imperial Terminal|          Arrival|              Domestic|            490|
|05/01/2014 12:00:...|01/01/2006 12:00:...|Imperial Terminal|        Departure|              Domestic|            498|
|05/01/2014 12:00:...|01/01/2006 12:00:...|   Misc. Terminal|          Arrival|              Domestic|            753|
|05/01/2014 12:00:...|01/01/2006 12:00:...|   Misc. Terminal|        Departure|              Domestic|            688|
|05/01/2014 12:00:...|01/01/2006 12:00:...|       Terminal 1|          Arrival|              Domestic|         401535|
|05/01/2014 12:00:...|01/01/2006 12:00:...|     

In [78]:
terminals = ['Terminal 1', 'Terminal 2', 'Terminal 3', 'Terminal 4', 'Terminal 5', 'Terminal 6', 'Terminal 7', 'Terminal 8', 'Tom Bradley International Terminal']


In [79]:
filtered_df = terminal_df.filter(col("Terminal").isin(terminals)).select('ReportPeriod', 'Passenger_Count', 'Terminal')



In [80]:
filtered_df.show(10)

+--------------------+---------------+----------+
|        ReportPeriod|Passenger_Count|  Terminal|
+--------------------+---------------+----------+
|01/01/2006 12:00:...|         401535|Terminal 1|
|01/01/2006 12:00:...|         389745|Terminal 1|
|01/01/2006 12:00:...|            561|Terminal 1|
|01/01/2006 12:00:...|          98991|Terminal 2|
|01/01/2006 12:00:...|         163067|Terminal 2|
|01/01/2006 12:00:...|          93672|Terminal 2|
|01/01/2006 12:00:...|         156751|Terminal 2|
|01/01/2006 12:00:...|         121649|Terminal 3|
|01/01/2006 12:00:...|          26585|Terminal 3|
|01/01/2006 12:00:...|         120111|Terminal 3|
+--------------------+---------------+----------+
only showing top 10 rows



In [81]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

In [82]:
df = filtered_df.withColumn("MonthYear", date_format(to_timestamp(col("ReportPeriod"), "MM/dd/yyyy HH:mm:ss"), "MMM yyyy"))

In [83]:
df.show(5)

+--------------------+---------------+----------+---------+
|        ReportPeriod|Passenger_Count|  Terminal|MonthYear|
+--------------------+---------------+----------+---------+
|01/01/2006 12:00:...|         401535|Terminal 1| Jan 2006|
|01/01/2006 12:00:...|         389745|Terminal 1| Jan 2006|
|01/01/2006 12:00:...|            561|Terminal 1| Jan 2006|
|01/01/2006 12:00:...|          98991|Terminal 2| Jan 2006|
|01/01/2006 12:00:...|         163067|Terminal 2| Jan 2006|
+--------------------+---------------+----------+---------+
only showing top 5 rows



In [84]:
monthly_passenger_count = df.groupBy("MonthYear").agg(sum("Passenger_Count").alias("TotalPassengers")) 

In [85]:
monthly_passenger_count.show(10)

+---------+---------------+
|MonthYear|TotalPassengers|
+---------+---------------+
| Jul 2016|        7621225|
| Nov 2013|        4914225|
| Feb 2011|        4121301|
| May 2007|        5256763|
| Apr 2017|        6640015|
| Apr 2008|        4949007|
| Dec 2014|        5825265|
| May 2016|        6571091|
| Aug 2011|        5918484|
| Sep 2012|        5053923|
+---------+---------------+
only showing top 10 rows



In [86]:
df.createOrReplaceTempView("filtered_terminals")

In [87]:
monthly_passenger_count = spark.sql("SELECT MonthYear, SUM(Passenger_Count) AS TotalPassengers FROM filtered_terminals GROUP BY MonthYear")

In [88]:
monthly_passenger_count.show(10)

+---------+---------------+
|MonthYear|TotalPassengers|
+---------+---------------+
| Jul 2016|        7621225|
| Nov 2013|        4914225|
| Feb 2011|        4121301|
| May 2007|        5256763|
| Apr 2017|        6640015|
| Apr 2008|        4949007|
| Dec 2014|        5825265|
| May 2016|        6571091|
| Aug 2011|        5918484|
| Sep 2012|        5053923|
+---------+---------------+
only showing top 10 rows



In [70]:
monthly_passenger_count.write.csv("/home/maryam/content/passenger_count-spark.sql.csv", header=True)

In [89]:
spark.stop()